In [1]:
import tensorflow as tf

In [6]:
@tf.function
def formula(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x


function_that_uses_a_graph = tf.function(formula) #  приймає у якості аргумента звичайну функцію і генерує обчислювальний граф

x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = formula(x1, y1, b1).numpy()
tf_function_value = function_that_uses_a_graph(x1, y1, b1).numpy() # тензор, який буде перетворено на багатовимірний масив numpy

assert(orig_value == tf_function_value)

print(x1.shape)
print(y1.shape)
print(b1.shape)



(1, 2)
(2, 1)
()


# **для того, щоб отримати тензор маючи багатовимірний масив numpy достатньо виконати наступний код:**

In [5]:
import numpy as np
import tensorflow as tf


x_array = np.array([1, 2, 3])
x_tensor = tf.convert_to_tensor(x_array)



# **При цьому перетворення тензора на багатовимірний масив numpy виглядає ще простіше:**

In [ ]:
x_tensor.numpy()


# **Розглянемо ще один приклад. У нейронних мережах часто використовується активаційна функція ReLU: f(x)=max(x,0)**

In [7]:
import tensorflow as tf

@tf.function
def relu_activation(x):
    if tf.greater(x, 0):
        return x
    return 0

print(relu_activation(tf.constant(1)).numpy())
print(relu_activation(tf.constant(-1)).numpy())


1
0


# **Автоматичне диференціювання**

def f(x):
    return 1 / x ** 2

### ***обчислення градієнтів***

***Тепер похідну можна обчислити так:***

In [10]:
import tensorflow as tf


def f(x):
    return 1 / x ** 2


x = tf.Variable(2.0)
with tf.GradientTape() as tape:
    y = f(x)
    dydx = tape.gradient(y, x)
    print(dydx)


tf.Tensor(-0.25, shape=(), dtype=float32)


За допомогою класу GradientTape tensorflow записує всі операції, необхідні для обчислення похідної в змінну tape.

Для того, щоб ***обчислити значення похідної в конкретній точці***, потрібно ***викликати метод gradient***, приймає 2 аргументи. Перший - функція, похідну якої ми хочемо обчислити. Другий - аргумент, за яким обчислюється похідна (причому це може бути тензор). Цей механізм нам ще знадобиться далі.



# **Створення нейронної мережі**

***Нехай потрібно написати примітивну нейронну мережу, яка обчислює значення виразу w⋅x+b.***

In [11]:
import tensorflow as tf


class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(5.0)
        self.b = tf.Variable(5.0)

    def __call__(self, x):
        return self.w * x + self.b


simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))


<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

***Тепер ми можемо спробувати створити складнішу нейронну мережу. Нехай потрібно створити нейромережу, в якій 2 шари. На першому шарі має 3 нейрони з трьома входами, на другому - 1, в якості активаційної функції використовуватимемо ReLU.***

In [12]:
class DenseLayer(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]), name="w"
        )
        self.b = tf.Variable(tf.zeros([out_features]), name="b")

    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)


class NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.layer_1 = DenseLayer(in_features=3, out_features=3)
    self.layer_2 = DenseLayer(in_features=3, out_features=1)

  def __call__(self, x):
    x = self.layer_1(x)
    return self.layer_2(x)


nn = NN(name="neural_network")
print("Results:", nn(tf.constant([[2.0, 2.0, 2.0]])))


Results: tf.Tensor([[0.42057833]], shape=(1, 1), dtype=float32)


***1. Для початку ми створили клас DenseLayer, в якому в загальному вигляді описали перетворення, що відбуваються в цьому шарі:***

# ***Навчання нейронної мережі***

***Створимо клас для даної моделі.***

In [13]:
import tensorflow as tf


class LinearModel(tf.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.w = tf.Variable(5.0)
        self.b = tf.Variable(0.0)

    def __call__(self, x):
        return self.w * x + self.b


***Тепер визначимо дві функції: loss та train. Одна буде обчислювати помилку, а друга - підлаштовувати ваги.***

In [14]:
def loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(target_y - predicted_y))


def train(model, x, y, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(y, model(x))
        dw, db = t.gradient(current_loss, [model.w, model.b])
        model.w.assign_sub(learning_rate * dw)
        model.b.assign_sub(learning_rate * db)


***Для оновлення ваг ми використовуємо метод assign_sub, який веде себе як оператор -=.***

In [15]:
def training_loop(model, x, y):
    for epoch in range(10):
        train(model, x, y, learning_rate=0.1)
        current_loss = loss(y, model(x))
        print(f"loss: {current_loss}")


***Для того щоб протестувати навчання моделі згенеруємо тестові дані***

In [16]:
TRUE_W = 3.0
TRUE_B = 2.0

NUM_EXAMPLES = 1000

x = tf.random.normal(shape=[NUM_EXAMPLES])
noise = tf.random.normal(shape=[NUM_EXAMPLES])
y = x * TRUE_W + TRUE_B + noise


***Тепер зберемо все разом.***

In [17]:
linear_model = LinearModel()
training_loop(linear_model, x, y)


loss: 5.876070499420166
loss: 4.220036029815674
loss: 3.1278774738311768
loss: 2.4074106216430664
loss: 1.932019829750061
loss: 1.6182626485824585
loss: 1.411134958267212
loss: 1.2743676900863647
loss: 1.184039831161499
loss: 1.1243698596954346
